## Step 1 : Imports

In [1]:
import pandas as pd

## Step 2: Load the oof files

In [2]:
fold_df = pd.read_csv("/kaggle/input/folding-strategy-rna/rna_dataset_with_fold.csv")
fold_df = fold_df[fold_df["fold"]==1]
fold_df.shape

(178, 7)

In [3]:
fold_df.columns

Index(['target_id', 'sequence', 'temporal_cutoff', 'description',
       'all_sequences', 'sequence_length', 'fold'],
      dtype='object')

In [4]:
fold_df

,target_id,sequence,temporal_cutoff,description,all_sequences,sequence_length,fold
67,1FFK_9,UUAGGCGGCCACAGCGGUGGGGUUGCCUCCCGUACCCAUCCCGAAC...,2000-08-14,CRYSTAL STRUCTURE OF THE LARGE RIBOSOMAL SUBUN...,>1FFK_1|Chain A[auth 0]|23S RRNA|Haloarcula ma...,122,1
74,1FOQ_A,UCAAUGGUACGGUACUUCCAUUGUCAUGUGUAUGUUGGGGAUUAAA...,2000-12-22,PENTAMERIC MODEL OF THE BACTERIOPHAGE PHI29 PR...,">1FOQ_1|Chains A, B, C, D, E|BACTERIOPHAGE PHI...",120,1
148,1S9S_A,GGCGGUACUAGUUGAGAAACUAGCUCUGUAUCUGGCGGACCCGUGG...,2004-04-06,SOLUTION STRUCTURE OF MLV PSI SITE,>1S9S_1|Chain A|MLV Psi encapsidation site|nul...,101,1
173,1ZC8_Z,GCCCGUCACGCCAUGGGAGCGGGCUCUACCCGAAGUCGCCGGGAGC...,2005-04-19,"Coordinates of tmRNA, SmpB, EF-Tu and h44 fitt...",>1ZC8_1|Chain A|TLD 16S ribosomal RNA|Thermus ...,91,1
186,1YSH_B,UAGACGGUGGGAGAGGGUGGUGGAAUUCCCGGAGUAGCGGUGAAAU...,2005-07-05,Localization and dynamic behavior of ribosomal...,>1YSH_1|Chain A|RNA (28-MER)|synthetic constru...,101,1
...,...,...,...,...,...,...,...
832,9G7C_A,GGGAGUCGGGUCUGUGGUUGAAAGUCGAUGCCAGUCGCAGGCGAAA...,2024-10-30,Cryo-EM structure of raiA ncRNA from Clostridi...,>9G7C_1|Chain A|raiA (224-MER)|Clostridium sp....,224,1
839,8T3E_EC,AAACUCCAUGUAUUGGUUACCCAUCUGCAUCGAAAACUCUCCGAAC...,2024-12-11,Hypomethylated yeast 80S bound with Taura synd...,>8T3E_1|Chain A[auth BA]|40S ribosomal protein...,202,1
840,8T3F_EC,AAACUCCAUGUAUUGGUUACCCAUCUGCAUCGAAAACUCUCCGAAC...,2024-12-11,Hypomethylated yeast 80S bound with Taura synd...,>8T3F_1|Chain A[auth BA]|40S ribosomal protein...,202,1
842,8Z1G_T,GGUAAAAUGGCUGAGUGAAGCAUUGGACUGUAAAUCUAAAGACAGG...,2024-12-18,Cryo-EM structure of human ELAC2-pre-tRNA,>8Z1G_1|Chain A[auth T]|Homo sapiens mitochond...,86,1


In [5]:
exp_1_oof = pd.read_csv("/kaggle/input/train-ribonanzanet-inference-03/ribonanzanet1_oof.csv")
exp_2_oof = pd.read_csv("/kaggle/input/ribonanzanet-3d-inference/ribonanzanet1_public_oof.csv")
exp_3_oof = pd.read_csv("/kaggle/input/ribonanzanet2-ddpm-inference/ribonanzanet2_oof.csv")
exp_4_oof = pd.read_csv("/kaggle/input/train-rhofold-inference/RhoFold/rhofold_01_oof.csv")

In [6]:
exp_1_oof["target_id"] = exp_1_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
exp_2_oof["target_id"] = exp_2_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
exp_3_oof["target_id"] = exp_3_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
exp_4_oof["target_id"] = exp_4_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])

In [7]:
exp_1_oof.shape, exp_2_oof.shape, exp_3_oof.shape, exp_4_oof.shape

((59508, 24), (59508, 24), (25713, 22), (25666, 23))

In [8]:
exp_1_oof.columns, exp_2_oof.columns, exp_3_oof.columns

(Index(['Unnamed: 0', 'ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'x_2',
        'y_2', 'z_2', 'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5',
        'z_5', 'flag', 'x_1_true', 'y_1_true', 'z_1_true', 'target_id'],
       dtype='object'),
 Index(['Unnamed: 0', 'ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'x_2',
        'y_2', 'z_2', 'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5',
        'z_5', 'flag', 'x_1_true', 'y_1_true', 'z_1_true', 'target_id'],
       dtype='object'),
 Index(['ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2',
        'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5', 'z_5',
        'x_1_true', 'y_1_true', 'z_1_true', 'target_id'],
       dtype='object'))

In [9]:
exp_1_oof.head()

,Unnamed: 0,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,...,y_4,z_4,x_5,y_5,z_5,flag,x_1_true,y_1_true,z_1_true,target_id
0,0,1SCL_A_1,G,1,-89.44650,440.98752,-191.07695,-85.431590,441.04114,-191.89421,...,440.77075,-189.97743,-88.49204,443.50134,-189.03722,False,13.760,-25.974001,0.102,1SCL_A
1,1,1SCL_A_2,G,2,-86.84754,444.70746,-187.42662,-82.974150,444.25116,-188.79414,...,445.63367,-186.70834,-86.35016,448.01245,-185.88399,False,9.310,-29.638000,2.669,1SCL_A
2,2,1SCL_A_3,G,3,-84.42002,450.20407,-186.49513,-80.990265,448.82376,-186.72595,...,450.10754,-184.92595,-84.62899,453.17456,-185.34094,False,5.529,-27.813000,5.878,1SCL_A
3,3,1SCL_A_4,U,4,-82.65829,454.56104,-187.05147,-80.184710,455.09888,-187.27373,...,455.17950,-185.91920,-83.52623,458.62033,-186.64482,False,2.678,-24.900999,9.793,1SCL_A
4,4,1SCL_A_5,G,5,-82.31447,459.51068,-189.50598,-80.920364,459.51350,-188.50897,...,460.33838,-188.98530,-84.19373,462.99750,-190.29022,False,1.827,-20.136000,11.793,1SCL_A


In [10]:
# Find common target_ids across all three dataframes
common_target_ids = set(exp_1_oof['target_id']).intersection(
    set(exp_2_oof['target_id'])).intersection(
    set(exp_3_oof['target_id'])).intersection(
    set(exp_4_oof['target_id'])).intersection(
    set(fold_df['target_id']))

# Filter exp_1_oof to keep only rows with common target_ids
exp_1_oof = exp_1_oof[exp_1_oof['target_id'].isin(common_target_ids)]
exp_2_oof = exp_2_oof[exp_2_oof['target_id'].isin(common_target_ids)]
exp_3_oof = exp_3_oof[exp_3_oof['target_id'].isin(common_target_ids)]
exp_4_oof = exp_4_oof[exp_4_oof['target_id'].isin(common_target_ids)]

In [11]:
len(common_target_ids)

173

In [12]:
exp_1_oof.shape, exp_2_oof.shape, exp_3_oof.shape, exp_4_oof.shape

((23230, 24), (23230, 24), (25124, 22), (25077, 23))

In [13]:
import pandas as pd

# Define key columns for matching
key_cols = ['ID', 'resname', 'resid', 'target_id']

# Find common keys across all three DataFrames
common_keys = pd.merge(exp_1_oof[key_cols], exp_2_oof[key_cols], on=key_cols)
common_keys = pd.merge(common_keys, exp_3_oof[key_cols], on=key_cols)
common_keys = pd.merge(common_keys, exp_4_oof[key_cols], on=key_cols)

# Filter each DataFrame using common keys
exp_1_oof = pd.merge(exp_1_oof, common_keys, on=key_cols)
exp_2_oof = pd.merge(exp_2_oof, common_keys, on=key_cols)
exp_3_oof = pd.merge(exp_3_oof, common_keys, on=key_cols)
exp_4_oof = pd.merge(exp_4_oof, common_keys, on=key_cols)

In [14]:
exp_1_oof.shape, exp_2_oof.shape, exp_3_oof.shape, exp_4_oof.shape

((23199, 24), (23199, 24), (23199, 22), (23199, 23))

In [15]:
exp_1_oof.attrs["name"] = "RINET1"
exp_2_oof.attrs["name"] = "RINET0"
exp_3_oof.attrs["name"] = "RINET2"
exp_4_oof.attrs["name"] = "RhoFold"

## Step -1: Hill climbing code

In [16]:
import gc
import os
import re
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing

In [17]:
def setup_usalign():
    """
    Sets up USalign by copying it to the working directory and making it executable.
    """
    # Copy USalign to working directory if it doesn't exist
    if not os.path.exists('/kaggle/working/USalign'):
        os.system('cp //kaggle/input/usalign/USalign /kaggle/working/')
        os.system('sudo chmod u+x /kaggle/working/USalign')
        print("USalign binary copied and made executable")
    return '/kaggle/working/USalign'

In [18]:
def parse_tmscore_output(output):
    # Extract TM-score based on length of reference structure (second)
    tm_score_match = re.findall(r'TM-score=\s+([\d.]+)', output)[1]
    if not tm_score_match:
        raise ValueError('No TM score found')
    return float(tm_score_match)


def write_target_line(
    atom_name, atom_serial, residue_name, chain_id, residue_num, x_coord, y_coord, z_coord, occupancy=1.0, b_factor=0.0, atom_type='P'
) -> str:
    """
    Writes a single line of PDB format based on provided atom information.
    """
    return f'ATOM  {atom_serial:>5d}  {atom_name:<5s} {residue_name:<3s} {residue_num:>3d}    {x_coord:>8.3f}{y_coord:>8.3f}{z_coord:>8.3f}{occupancy:>6.2f}{b_factor:>6.2f}           {atom_type}\n'

In [19]:
def write2pdb(df: pd.DataFrame, xyz_id: str, target_path: str) -> int:
    resolved_cnt = 0
    with open(target_path, 'w') as target_file:
        for _, row in df.iterrows():
            x_coord = row[f'x_{xyz_id}']
            y_coord = row[f'y_{xyz_id}']
            z_coord = row[f'z_{xyz_id}']

            if x_coord > -1e17 and y_coord > -1e17 and z_coord > -1e17:
                resolved_cnt += 1
                target_line = write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x_coord,
                    y_coord=y_coord,
                    z_coord=z_coord,
                    atom_type='C',
                )
                target_file.write(target_line)
    return resolved_cnt

In [20]:
def write_ground_truth_pdb(df: pd.DataFrame, target_path: str) -> int:
    """
    Writes ground truth coordinates to a PDB file.
    """
    resolved_cnt = 0
    with open(target_path, 'w') as target_file:
        for _, row in df.iterrows():
            x_coord = row['x_1_true']
            y_coord = row['y_1_true']
            z_coord = row['z_1_true']

            if x_coord > -1e17 and y_coord > -1e17 and z_coord > -1e17:
                resolved_cnt += 1
                target_line = write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x_coord,
                    y_coord=y_coord,
                    z_coord=z_coord,
                    atom_type='C',
                )
                target_file.write(target_line)
    return resolved_cnt

In [21]:
def evaluate_single_weight(args):
    """
    Evaluates a single weight combination for ensembling two RNA structure predictions.
    
    Args:
        args (tuple): Contains weight, first_pred_df, second_pred_df, temporary_dir, and usalign_path
        
    Returns:
        tuple: weight and resulting score
    """
    weight, first_pred_df, second_pred_df, temp_dir, usalign_path = args
    
    # Create a copy of the first prediction dataframe
    ensemble_df = first_pred_df.copy()
    
    # For each prediction (1-5), blend the coordinates using the weight
    for pred_idx in range(1, 6):
        # Blend x coordinates
        ensemble_df[f'x_{pred_idx}'] = (1 - weight) * first_pred_df[f'x_{pred_idx}'] + weight * second_pred_df[f'x_{pred_idx}']
        # Blend y coordinates
        ensemble_df[f'y_{pred_idx}'] = (1 - weight) * first_pred_df[f'y_{pred_idx}'] + weight * second_pred_df[f'y_{pred_idx}']
        # Blend z coordinates
        ensemble_df[f'z_{pred_idx}'] = (1 - weight) * first_pred_df[f'z_{pred_idx}'] + weight * second_pred_df[f'z_{pred_idx}']
    
    # Create unique filenames for this process to avoid conflicts in parallel processing
    process_id = os.getpid()
    native_pdb = f'{temp_dir}/native_{process_id}.pdb'
    predicted_pdb = f'{temp_dir}/predicted_{process_id}.pdb'
    
    # Evaluate using TM-score
    score = score_ensemble(ensemble_df, native_pdb, predicted_pdb, usalign_path)
    
    return weight, score

In [22]:
from tqdm import tqdm

def score_ensemble(ensemble_df, native_pdb='native.pdb', predicted_pdb='predicted.pdb', usalign_path=None):
    """
    Scores the ensemble predictions against the ground truth using TM-score.
    
    Args:
        ensemble_df (pd.DataFrame): DataFrame with predicted coordinates and ground truth
        native_pdb (str): Path to write the native PDB file
        predicted_pdb (str): Path to write the predicted PDB file
        usalign_path (str): Path to the USalign executable
        
    Returns:
        float: Average TM-score across all targets
    """
    if usalign_path is None:
        usalign_path = '/kaggle/working/USalign'
    
    from contextlib import contextmanager
    
    @contextmanager
    def no_stdout():
        old_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        try:
            yield
        finally:
            sys.stdout = old_stdout
    
    # Extract target_id from ID (target_resid)
    ensemble_df['target_id'] = ensemble_df['ID'].apply(lambda x: x.split('_')[0])
    
    results = []
    # Iterate through each target_id and compute TM-score
    unique_targets = ensemble_df['target_id'].unique()
    for target_id in tqdm(unique_targets):
        group_data = ensemble_df[ensemble_df['target_id'] == target_id]
        
        # Write ground truth PDB once per target
        with no_stdout():
            resolved_cnt_truth = write_ground_truth_pdb(group_data, native_pdb)
        
        if resolved_cnt_truth == 0:
            continue  # Skip if no ground truth coordinates
        
        target_id_scores = []
        for pred_cnt in range(1, 6):
            # Write predicted PDB
            with no_stdout():
                resolved_cnt_pred = write2pdb(group_data, pred_cnt, predicted_pdb)
            
            if resolved_cnt_pred > 0:
                try:
                    command = f'{usalign_path} {predicted_pdb} {native_pdb} -atom " C1\'"'
                    usalign_output = os.popen(command).read()
                    tm_score = parse_tmscore_output(usalign_output)
                    target_id_scores.append(tm_score)
                except Exception as e:
                    # Silent error handling during optimization
                    continue
        
        if target_id_scores:
            results.append(max(target_id_scores))
    
    # Return average TM-score across all targets
    if not results:
        return 0.0
    return float(sum(results) / len(results))

In [23]:
def hill_climbing_optimizer(experiments_list, weight_step=0.05, max_iterations=1, num_cores=None):
    """
    Performs hill climbing optimization to find the best ensemble weights for RNA structure predictions.
    
    Args:
        experiments_list (list): List of dataframes with predictions
        weight_step (float): Step size for weight exploration
        max_iterations (int): Maximum number of iterations
        num_cores (int): Number of CPU cores to use for parallelization
        
    Returns:
        tuple: Dictionary of weights, best score, and final ensemble dataframe
    """
    # Set up USalign
    usalign_path = setup_usalign()
    
    if num_cores is None:
        num_cores = max(1, multiprocessing.cpu_count() - 1)
    
    # Create temporary directory for parallel processing
    temp_dir = '/tmp/rna_ensemble'
    os.makedirs(temp_dir, exist_ok=True)
    
    # Start with the first experiment as baseline
    best_ensemble_df = experiments_list[0].copy()
    print(f"First model in ensemble : {best_ensemble_df.attrs['name']}")
    remaining_experiments = experiments_list[1:].copy()
    
    # Dictionary to store weights for each experiment
    model_weights = {}
    
    # Define the weight range to explore
    weights_range = np.arange(0.0, 1.0 + weight_step, weight_step)
    weights_range = np.arange(-0.50, 0.51, 0.01)
    print(f"Number of weights testing : {len(weights_range)}")
    
    # Get initial score
    initial_score = score_ensemble(best_ensemble_df, usalign_path=usalign_path)
    best_score = initial_score
    print(f"Initial score: {best_score:.6f}")
    
    iteration = 0
    while remaining_experiments and iteration < max_iterations:
        iteration += 1
        print(f"\nIteration {iteration}/{max_iterations}")
        
        best_iteration = {
            'index': -1,
            'weight': 0,
            'score': best_score,
            'df': None
        }
        
        # Try each remaining model
        for i, model_df in enumerate(remaining_experiments):
            try:
                print(f"Testing model {i+1}/{len(remaining_experiments)}")
                
                # Prepare arguments for parallel processing
                args_list = [(weight, best_ensemble_df, model_df, temp_dir, usalign_path) 
                             for weight in weights_range]
                
                # Process weights in parallel
                with ProcessPoolExecutor(max_workers=num_cores) as executor:
                    futures = [executor.submit(evaluate_single_weight, args) for args in args_list]
                    
                    # Process results as they complete
                    for future in tqdm(as_completed(futures), total=len(futures), 
                                      desc=f"Testing weights for model {model_df.attrs['name']}"):
                        weight, new_score = future.result()
                        
                        # Update best if improved
                        if new_score > best_iteration['score']:
                            # Create the ensemble for this weight
                            tmp_ensemble_df = best_ensemble_df.copy()
                            for pred_idx in range(1, 6):
                                tmp_ensemble_df[f'x_{pred_idx}'] = (1 - weight) * best_ensemble_df[f'x_{pred_idx}'] + weight * model_df[f'x_{pred_idx}']
                                tmp_ensemble_df[f'y_{pred_idx}'] = (1 - weight) * best_ensemble_df[f'y_{pred_idx}'] + weight * model_df[f'y_{pred_idx}']
                                tmp_ensemble_df[f'z_{pred_idx}'] = (1 - weight) * best_ensemble_df[f'z_{pred_idx}'] + weight * model_df[f'z_{pred_idx}']
                            
                            best_iteration.update({
                                'index': i,
                                'weight': weight,
                                'score': new_score,
                                'df': tmp_ensemble_df.copy(),
                                'model_added' : model_df.attrs["name"]
                            })
                
                # Clean up
                gc.collect()
                
            except Exception as e:
                print(f"Error processing model {i+1}: {str(e)}")
                continue
        
        # Check if we found an improvement
        if best_iteration['index'] == -1:
            print("No improvement found, stopping")
            break
            
        # Update ensemble with best model found
        best_score = best_iteration['score']
        best_ensemble_df = best_iteration['df'].copy()
        
        # Add weight to the selected model and remove it from remaining
        selected_model = remaining_experiments.pop(best_iteration['index'])
        model_weights[f"model_{best_iteration['model_added']}"] = best_iteration['weight']
        
        print(f"Added model {best_iteration['model_added']} with weight {best_iteration['weight']:.4f}, New score: {best_iteration['score']:.6f}")
        
    print("\nOptimization complete!")
    print(f"Initial score: {initial_score:.6f}")
    print(f"Final score: {best_score:.6f}")
    print(f"Improvement: {best_score - initial_score:.6f} ({(best_score - initial_score) / initial_score * 100:.2f}%)")
    print(f"Model weights: {model_weights}")
    
    # Clean up temporary directory
    for filename in os.listdir(temp_dir):
        os.remove(os.path.join(temp_dir, filename))
    os.rmdir(temp_dir)
    
    return model_weights, best_score, best_ensemble_df

In [24]:
# Create list of experiment dataframes
experiments_list = [exp_1_oof, exp_2_oof, exp_3_oof, exp_4_oof]

# Run the hill climbing optimizer
model_weights, best_score, final_ensemble = hill_climbing_optimizer(
    experiments_list=experiments_list,
    weight_step=0.1,
    max_iterations=10,
    num_cores=4
)

USalign binary copied and made executable
First model in ensemble : RINET1
Number of weights testing : 101


 43%|████▎     | 67/155 [00:24<00:13,  6.43it/s]Sequence is too short <3!: predicted.pdb
Sequence is too short <3!: predicted.pdb
Sequence is too short <3!: predicted.pdb
Sequence is too short <3!: predicted.pdb
Sequence is too short <3!: predicted.pdb
100%|██████████| 155/155 [00:56<00:00,  2.73it/s]

Initial score: 0.358453

Iteration 1/10
Testing model 1/3



 40%|████      | 62/155 [00:34<00:32,  2.89it/s]Sequence is too short <3!: /tmp/rna_ensemble/predicted_1594.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_1594.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_1594.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_1594.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_1594.pdb
 44%|████▍     | 68/155 [00:36<00:18,  4.60it/s]Sequence is too short <3!: /tmp/rna_ensemble/predicted_1595.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_1595.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_1595.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_1597.pdb
Sequence is too short <3!: /tmp/rna_ensemble/predicted_1595.pdb
Segmentation fault (core dumped)

Testing model 2/3


...

Testing model 2/2



 43%|████▎     | 67/155 [00:35<00:22,  3.92it/s]Sequence is too short <3!: /tmp/rna_ensemble/predicted_753010.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_753010.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_753010.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_753010.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_753010.pdb
Sequence is too short <3!: /tmp/rna_ensemble/predicted_753012.pdb
 41%|████▏     | 64/155 [00:35<00:29,  3.07it/s]Segmentation fault (core dumped)
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_753012.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_753012.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_753012.pdb
Segmentation fault (core dumped)
Sequence is 

Added model RINET2 with weight 0.0100, New score: 0.362408

Iteration 3/10
Testing model 1/1



 44%|████▍     | 68/155 [00:34<00:20,  4.22it/s]Sequence is too short <3!: /tmp/rna_ensemble/predicted_940892.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_940892.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_940892.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_940892.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_940892.pdb
Segmentation fault (core dumped)
 43%|████▎     | 66/155 [00:34<00:25,  3.53it/s]Sequence is too short <3!: /tmp/rna_ensemble/predicted_940895.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_940895.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_940895.pdb
Segmentation fault (core dumped)
Sequence is too short <3!: /tmp/rna_ensemble/predicted_940895.pdb
Segmentation fault (core dumped)
Sequence is 

No improvement found, stopping

Optimization complete!
Initial score: 0.358453
Final score: 0.362408
Improvement: 0.003955 (1.10%)
Model weights: {'model_RINET0': 0.010000000000000453, 'model_RINET2': 0.010000000000000453}


In [25]:
model_weights, best_score, final_ensemble

({'model_RINET0': 0.010000000000000453, 'model_RINET2': 0.010000000000000453},
 0.3624084415584414,
        Unnamed: 0         ID resname  resid         x_1         y_1  \
 0            1720   1FFK_9_1       U      1 -103.988279  467.668598   
 1            1721   1FFK_9_2       U      2 -102.507799  464.748296   
 2            1722   1FFK_9_3       A      3  -99.732552  461.893026   
 3            1723   1FFK_9_4       G      4 -104.592810  457.038302   
 4            1724   1FFK_9_5       G      5 -107.744409  451.865816   
 ...           ...        ...     ...    ...         ...         ...   
 23194       64413  8Z1F_T_62       U     62  -90.337759  452.430621   
 23195       64414  8Z1F_T_63       A     63  -86.829195  449.249144   
 23196       64415  8Z1F_T_64       C     64  -86.630040  450.184442   
 23197       64416  8Z1F_T_65       C     65  -87.150484  445.196258   
 23198       64417  8Z1F_T_66       A     66  -90.105690  441.404682   
 
               z_1         x_2    

In [26]:
# Save the final ensemble
final_ensemble.to_csv('final_ensemble.csv', index=False)